Imports

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
builder = tfds.builder("cats_vs_dogs")
builder.download_and_prepare()

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]WARNING:absl:1738 images were corrupted and were skipped
                                                                        

Dataset cats_vs_dogs downloaded and prepared to C:\Users\manup\tensorflow_datasets\cats_vs_dogs\4.0.0. Subsequent calls will reuse this data.


In [5]:

train_data = builder.as_dataset(split="train")

for example in train_data.take(5):
    image = example["image"]
    label = example["label"]
    filename = example["image/filename"]
    print(f"Label: {label.numpy()}, Filename: {filename.numpy()}")

Label: 1, Filename: b'PetImages\\Dog\\10396.jpg'
Label: 1, Filename: b'PetImages\\Dog\\4077.jpg'
Label: 1, Filename: b'PetImages\\Dog\\10497.jpg'
Label: 0, Filename: b'PetImages\\Cat\\2763.jpg'
Label: 1, Filename: b'PetImages\\Dog\\11257.jpg'


In [6]:
# Prepare the data for training and testing
def preprocess_data(data):
    def _preprocess(example):
        image = tf.image.resize(example["image"], (224, 224))
        image = tf.image.convert_image_dtype(image, tf.float32)
        label = tf.one_hot(example["label"], depth=2)
        return image, label

    data = data.map(_preprocess)
    data = data.batch(32)
    data = data.prefetch(tf.data.experimental.AUTOTUNE)
    return data

In [7]:
train_size = int(0.8 * builder.info.splits["train"].num_examples)
test_size = builder.info.splits["train"].num_examples - train_size

train_data = train_data.take(train_size)
test_data = train_data.skip(train_size).take(test_size)

In [8]:
train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

In [9]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 5
TRAIN_DIR = train_data

In [10]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

94765736/94765736 [==============================] - 86s 1us/step


In [11]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(IMG_SIZE,IMG_SIZE,3),
                   pooling='avg',classes=2,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(2, activation='softmax'))

In [12]:
resnet_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 module_wrapper (ModuleWrapp  (None, 2048)             0         
 er)                                                             
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 dense_1 (Dense)             (None, 2)                 1026      
                                                                 
Total params: 24,637,826
Trainable params: 1,050,114
Non-trainable params: 23,587,712
_________________________________________________________________


In [13]:
resnet_model.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [20]:
history = resnet_model.fit(train_data, validation_data=test_data, epochs=11,verbose=0)
for epoch in range(0, 30, 10):
    train_accuracy = history.history['accuracy'][epoch]
    val_accuracy = history.history['val_accuracy'][epoch]
    print(f'Epoch {epoch + 1} - Training Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}')

In [ ]:
test_loss, test_accuracy = resnet_model.evaluate(test_data)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
fig1 = plt.gcf()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.axis(ymin=0.4,ymax=1)
plt.grid()
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'validation'])
plt.show()